In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse

In [3]:
dataset = fetch_california_housing()
x = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

# use infer_signature for setting the input and output schema
from mlflow.models.signature import infer_signature
signature = infer_signature(x_train, y_train)

mlflow.set_tracking_uri("http://localhost:5000")

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

with mlflow.start_run():
    grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_param("n_estimators", best_model.n_estimators)
    mlflow.log_param("max_depth", best_model.max_depth)
    mlflow.log_param("min_samples_split", best_model.min_samples_split)
    mlflow.log_param("min_samples_leaf", best_model.min_samples_leaf)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    # Model registry does not work with file store
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        # Model registered in MLflow model registry
        mlflow.sklearn.log_model(
            best_model,
            "model",
            registered_model_name="RandomForestRegressor",
            signature=signature,
        )
    else:
        # Model saved in local directory
        mlflow.sklearn.log_model(best_model, "model", signature=signature)
    
print(mse, r2)


2025/09/15 00:56:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/15 00:56:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'RandomForestRegressor'.
2025/09/15 00:56:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestRegressor, version 1
Created version '1' of model 'RandomForestRegressor'.


🏃 View run stylish-rook-590 at: http://localhost:5000/#/experiments/0/runs/6dd405ca1bad436fa4664577108ff0d2
🧪 View experiment at: http://localhost:5000/#/experiments/0
0.24562823156635621 0.8152913083433252
